# 📘 Day 2: The Transformer Architecture

**🎯 Goal:** Master the complete Transformer architecture - the foundation of ALL modern LLMs

**⏱️ Time:** 120-150 minutes

**🌟 Why This Matters for AI:**
- Transformers power EVERY modern AI: GPT-4, Claude, Gemini, ChatGPT, BERT, T5
- The "Attention Is All You Need" (2017) paper revolutionized AI
- Understanding Transformers = Understanding how ALL modern LLMs work
- From language to images: ViT, DALL-E, Stable Diffusion use transformers
- From coding to science: GitHub Copilot, AlphaFold use transformers
- The architecture that started the AI revolution (2017-2025)

**What You'll Build Today:**
- Complete Transformer encoder-decoder from scratch
- Understand GPT (decoder-only) vs BERT (encoder-only) vs T5 (encoder-decoder)
- Mini-transformer for sequence-to-sequence tasks
- See how ChatGPT architecture differs from BERT

---

## 🚀 The Transformer Revolution

**Before Transformers (Pre-2017):**
- **RNNs/LSTMs**: Slow, sequential, forget long contexts
- **CNNs**: Only local patterns, limited for sequences
- **Maximum performance**: Limited by architecture

**After Transformers (2017-2025):**
```
2017: "Attention Is All You Need" paper published
2018: BERT revolutionized NLP (bidirectional understanding)
2019: GPT-2 showed impressive text generation
2020: GPT-3 (175B params) - massive scale breakthrough
2022: ChatGPT launched - transformers meet RLHF
2023: GPT-4, Claude, Gemini - multimodal transformers
2024-2025: Every AI breakthrough uses transformers!
```

### 🎯 Why Transformers Won:

**1. Parallelization**
- RNN: Process words one-by-one → SLOW ❌
- Transformer: Process ALL words simultaneously → FAST ✅
- Result: Can train on billions of tokens efficiently

**2. Long-Range Dependencies**
- RNN: Forgets early context in long sequences ❌
- Transformer: Self-attention connects ALL positions ✅
- Result: Understands context from thousands of tokens away

**3. Scalability**
- RNN: Doesn't improve much with more parameters ❌
- Transformer: Scales beautifully to billions of parameters ✅
- Result: GPT-3 (175B), GPT-4 (1.7T+ estimated), Claude, Gemini

**4. Transfer Learning**
- Pre-train once on massive data
- Fine-tune for ANY task
- Result: BERT, GPT revolutionized NLP transfer learning

### 📊 Transformer Timeline:

| Year | Model | Type | Parameters | Impact |
|------|-------|------|------------|--------|
| 2017 | Transformer | Encoder-Decoder | 65M | Started revolution |
| 2018 | BERT | Encoder-only | 340M | NLP understanding |
| 2019 | GPT-2 | Decoder-only | 1.5B | Text generation |
| 2020 | GPT-3 | Decoder-only | 175B | Few-shot learning |
| 2020 | T5 | Encoder-Decoder | 11B | Text-to-text |
| 2022 | ChatGPT | Decoder-only | 175B+ | Conversational AI |
| 2023 | GPT-4 | Decoder-only | 1.7T+ | Multimodal AI |
| 2023 | Claude | Decoder-only | Unknown | Constitutional AI |
| 2024 | Gemini | Decoder-only | 1.5T+ | Native multimodal |

Let's build this revolutionary architecture!

In [ ]:
# Import essential libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Make plots beautiful
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print("Let's build the Transformer architecture from scratch! 🚀")

## 🏗️ Transformer Architecture Overview

**The Complete Picture:**

```
INPUT SEQUENCE (e.g., "The cat sat")
        ↓
    EMBEDDINGS + POSITIONAL ENCODING
        ↓
  ┌─────────────────────────────────┐
  │         ENCODER STACK           │
  │  ┌──────────────────────────┐  │
  │  │  Multi-Head Attention    │  │ ← Self-attention
  │  └──────────────────────────┘  │
  │           ↓                     │
  │  ┌──────────────────────────┐  │
  │  │  Feed Forward Network    │  │
  │  └──────────────────────────┘  │
  │    (Repeat N=6 times)          │
  └─────────────────────────────────┘
        ↓
  ENCODER OUTPUT (contextual representations)
        ↓
  ┌─────────────────────────────────┐
  │         DECODER STACK           │
  │  ┌──────────────────────────┐  │
  │  │ Masked Multi-Head Attn   │  │ ← Looks only at previous words
  │  └──────────────────────────┘  │
  │           ↓                     │
  │  ┌──────────────────────────┐  │
  │  │ Cross-Attention          │  │ ← Attends to encoder output
  │  └──────────────────────────┘  │
  │           ↓                     │
  │  ┌──────────────────────────┐  │
  │  │ Feed Forward Network     │  │
  │  └──────────────────────────┘  │
  │    (Repeat N=6 times)          │
  └─────────────────────────────────┘
        ↓
    LINEAR + SOFTMAX
        ↓
OUTPUT SEQUENCE (e.g., "Le chat était assis")
```

### 🎯 Key Components:

**1. Embeddings + Positional Encoding**
- Convert words to vectors
- Add position information
- *(Learned this in Day 1!)*

**2. Encoder Block** (Repeated N times)
- **Multi-Head Self-Attention**: Each word attends to all words
- **Feed-Forward Network**: Process each position independently
- **Layer Normalization**: Stabilize training
- **Residual Connections**: Skip connections for gradient flow

**3. Decoder Block** (Repeated N times)
- **Masked Self-Attention**: Can't peek at future words
- **Cross-Attention**: Attend to encoder outputs
- **Feed-Forward Network**: Same as encoder
- **Layer Norm + Residuals**: Same as encoder

**4. Output Layer**
- Linear projection to vocabulary size
- Softmax for probability distribution
- Predict next token

### 🔑 Critical Design Choices:

**Why Add & Norm?**
```
Output = LayerNorm(Input + Sublayer(Input))
```
- **Residual (+)**: Helps gradients flow through deep networks
- **LayerNorm**: Stabilizes training, enables deeper networks

**Why Feed-Forward Network?**
- Attention mixes information
- FFN processes each position independently
- Adds non-linearity and capacity

**Why Stack Multiple Layers?**
- Layer 1: Learn basic patterns ("cat" relates to "sat")
- Layer 6: Learn complex reasoning (coreference, inference)
- Deeper = More sophisticated understanding

Let's implement each component!

In [ ]:
# Positional Encoding (from Day 1, but let's implement it again)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        """
        Positional Encoding Layer
        
        Args:
            d_model: Embedding dimension
            max_len: Maximum sequence length
            dropout: Dropout probability
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Create positional encoding matrix
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term)
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Add batch dimension
        pe = pe.unsqueeze(0)  # (1, max_len, d_model)
        
        # Register as buffer (not a parameter, but should be saved/loaded)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        """
        Args:
            x: Input embeddings (batch_size, seq_len, d_model)
        
        Returns:
            x with positional encoding added
        """
        # Add positional encoding
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)

# Test positional encoding
d_model = 512
seq_len = 20
batch_size = 2

pos_enc = PositionalEncoding(d_model)
x = torch.randn(batch_size, seq_len, d_model)
output = pos_enc(x)

print("✅ Positional Encoding Test")
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"\n💡 Positional encoding adds position information to embeddings!")

In [ ]:
# Multi-Head Attention (improved version from Day 1)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        """
        Multi-Head Attention Layer
        
        Args:
            d_model: Model dimension (must be divisible by num_heads)
            num_heads: Number of attention heads
            dropout: Dropout probability
        """
        super(MultiHeadAttention, self).__init__()
        
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads  # Dimension per head
        
        # Linear projections for Q, K, V
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        
        # Output projection
        self.W_o = nn.Linear(d_model, d_model)
        
        self.dropout = nn.Dropout(dropout)
        
    def split_heads(self, x):
        """Split the last dimension into (num_heads, d_k)"""
        batch_size, seq_len, d_model = x.shape
        return x.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
    
    def combine_heads(self, x):
        """Inverse of split_heads"""
        batch_size, num_heads, seq_len, d_k = x.shape
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
    
    def forward(self, query, key, value, mask=None):
        """
        Args:
            query: (batch_size, seq_len_q, d_model)
            key: (batch_size, seq_len_k, d_model)
            value: (batch_size, seq_len_v, d_model)
            mask: Optional mask (batch_size, 1, seq_len_q, seq_len_k)
        
        Returns:
            output: (batch_size, seq_len_q, d_model)
            attention_weights: (batch_size, num_heads, seq_len_q, seq_len_k)
        """
        batch_size = query.shape[0]
        
        # Linear projections
        Q = self.W_q(query)  # (batch, seq_len_q, d_model)
        K = self.W_k(key)    # (batch, seq_len_k, d_model)
        V = self.W_v(value)  # (batch, seq_len_v, d_model)
        
        # Split into multiple heads
        Q = self.split_heads(Q)  # (batch, num_heads, seq_len_q, d_k)
        K = self.split_heads(K)  # (batch, num_heads, seq_len_k, d_k)
        V = self.split_heads(V)  # (batch, num_heads, seq_len_v, d_k)
        
        # Scaled dot-product attention
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask (if provided)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # Attention weights
        attention_weights = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)
        
        # Apply attention to values
        attention_output = torch.matmul(attention_weights, V)
        
        # Combine heads
        attention_output = self.combine_heads(attention_output)
        
        # Final linear projection
        output = self.W_o(attention_output)
        
        return output, attention_weights

# Test multi-head attention
d_model = 512
num_heads = 8
seq_len = 10
batch_size = 2

mha = MultiHeadAttention(d_model, num_heads)
x = torch.randn(batch_size, seq_len, d_model)

output, attn_weights = mha(x, x, x)

print("✅ Multi-Head Attention Test")
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"Attention weights shape: {attn_weights.shape}")
print(f"\n💡 {num_heads} heads learn different attention patterns!")

## 🍔 Feed-Forward Network

**Purpose:** Process each position independently with non-linear transformations

**Architecture:**
```
Input (d_model=512)
    ↓
Linear (512 → 2048)
    ↓
ReLU Activation
    ↓
Linear (2048 → 512)
    ↓
Dropout
    ↓
Output (d_model=512)
```

**Key Points:**
- Expands dimension by 4x (512 → 2048)
- Applies non-linearity (ReLU)
- Projects back to original dimension
- Applied to EACH position independently

**Why It Matters:**
- **Attention**: Mixes information across positions
- **FFN**: Adds depth and capacity to each position
- Together: Rich, deep representations!

In [ ]:
# Position-wise Feed-Forward Network

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        Position-wise Feed-Forward Network
        
        Args:
            d_model: Model dimension (e.g., 512)
            d_ff: Hidden dimension (typically 4 * d_model)
            dropout: Dropout probability
        """
        super(PositionWiseFeedForward, self).__init__()
        
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        """
        Args:
            x: Input (batch_size, seq_len, d_model)
        
        Returns:
            Output (batch_size, seq_len, d_model)
        """
        # Expand: d_model → d_ff
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        
        # Project back: d_ff → d_model
        x = self.fc2(x)
        
        return x

# Test feed-forward network
d_model = 512
d_ff = 2048  # 4x expansion
seq_len = 10
batch_size = 2

ffn = PositionWiseFeedForward(d_model, d_ff)
x = torch.randn(batch_size, seq_len, d_model)
output = ffn(x)

print("✅ Feed-Forward Network Test")
print(f"Input shape: {x.shape}")
print(f"Hidden dimension: {d_ff}")
print(f"Output shape: {output.shape}")
print(f"\n💡 FFN processes each position independently!")
print(f"   Expands to {d_ff} dims, then back to {d_model}")

## 🏗️ Encoder Block

**The Encoder Layer combines everything:**

```
Input
  ↓
  ├──→ Multi-Head Self-Attention
  │           ↓
  └─────→ Add & Norm
  ↓
  ├──→ Feed-Forward Network
  │           ↓
  └─────→ Add & Norm
  ↓
Output (to next encoder layer or decoder)
```

**Key Features:**
1. **Multi-Head Self-Attention**: Learn relationships between all words
2. **Residual Connection**: `output = input + sublayer(input)`
3. **Layer Normalization**: Stabilize training
4. **Feed-Forward Network**: Add depth and capacity
5. **Another Residual + LayerNorm**: More stable gradients

**Why Residuals?**
- Deep networks (6-96 layers) need gradient flow
- Residuals create shortcut paths
- Enables training very deep transformers (GPT-3: 96 layers!)

**Why LayerNorm?**
- Normalizes across feature dimension
- Stabilizes training
- Enables deeper, more stable networks

In [ ]:
# Transformer Encoder Block

class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        Transformer Encoder Block
        
        Args:
            d_model: Model dimension
            num_heads: Number of attention heads
            d_ff: Feed-forward hidden dimension
            dropout: Dropout probability
        """
        super(EncoderBlock, self).__init__()
        
        # Multi-head self-attention
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        
        # Feed-forward network
        self.ffn = PositionWiseFeedForward(d_model, d_ff, dropout)
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        """
        Args:
            x: Input (batch_size, seq_len, d_model)
            mask: Optional attention mask
        
        Returns:
            Output (batch_size, seq_len, d_model)
        """
        # Self-attention with residual connection and layer norm
        attn_output, _ = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))  # Add & Norm
        
        # Feed-forward with residual connection and layer norm
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))  # Add & Norm
        
        return x

# Test encoder block
d_model = 512
num_heads = 8
d_ff = 2048
seq_len = 10
batch_size = 2

encoder_block = EncoderBlock(d_model, num_heads, d_ff)
x = torch.randn(batch_size, seq_len, d_model)
output = encoder_block(x)

print("✅ Encoder Block Test")
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")
print(f"\n🏗️ Complete encoder block working!")
print(f"   - Multi-head self-attention ✅")
print(f"   - Residual connections ✅")
print(f"   - Layer normalization ✅")
print(f"   - Feed-forward network ✅")

## 🎯 Decoder Block

**The Decoder is more complex:**

```
Input (target sequence)
  ↓
  ├──→ Masked Multi-Head Self-Attention (can't see future)
  │           ↓
  └─────→ Add & Norm
  ↓
  ├──→ Cross-Attention (attend to encoder output)
  │           ↓
  └─────→ Add & Norm
  ↓
  ├──→ Feed-Forward Network
  │           ↓
  └─────→ Add & Norm
  ↓
Output (to next decoder layer or final prediction)
```

### 🎭 Key Differences from Encoder:

**1. Masked Self-Attention**
- **Why?** During training, we can't let decoder see future words!
- **Example:** Translating "The cat" → "Le chat"
  - When predicting "Le", can't see "chat" (future)
  - Mask prevents attention to future positions

**Masking Example:**
```
Sentence: [Le, chat, dort]

Attention matrix (before masking):
       Le  chat  dort
Le   [ 1    1     1  ]  ← Can see all (BAD!)
chat [ 1    1     1  ]  ← Can see all (BAD!)
dort [ 1    1     1  ]  ← Can see all (BAD!)

Attention matrix (after masking):
       Le  chat  dort
Le   [ 1    0     0  ]  ← Only sees itself ✅
chat [ 1    1     0  ]  ← Sees Le, chat ✅
dort [ 1    1     1  ]  ← Sees all previous ✅
```

**2. Cross-Attention**
- **Query**: From decoder (what I'm generating)
- **Key & Value**: From encoder (source sequence)
- **Purpose**: Align target with source
- **Example:** When generating "chat", attend strongly to "cat"

**3. Same Feed-Forward as Encoder**
- Identical architecture
- Separate parameters

### 🎯 This is How Translation Works!

**English → French:**
1. **Encoder**: Understands "The cat sat on the mat"
2. **Decoder**: Generates "Le chat était assis sur le tapis"
3. **Cross-Attention**: Links "cat" ↔ "chat", "mat" ↔ "tapis"

In [ ]:
# Transformer Decoder Block

class DecoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        Transformer Decoder Block
        
        Args:
            d_model: Model dimension
            num_heads: Number of attention heads
            d_ff: Feed-forward hidden dimension
            dropout: Dropout probability
        """
        super(DecoderBlock, self).__init__()
        
        # Masked multi-head self-attention
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        
        # Cross-attention (attend to encoder output)
        self.cross_attn = MultiHeadAttention(d_model, num_heads, dropout)
        
        # Feed-forward network
        self.ffn = PositionWiseFeedForward(d_model, d_ff, dropout)
        
        # Layer normalization
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        """
        Args:
            x: Decoder input (batch_size, tgt_seq_len, d_model)
            encoder_output: Encoder output (batch_size, src_seq_len, d_model)
            src_mask: Source mask for encoder-decoder attention
            tgt_mask: Target mask for masked self-attention
        
        Returns:
            Output (batch_size, tgt_seq_len, d_model)
        """
        # 1. Masked self-attention (can't see future)
        self_attn_output, _ = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(self_attn_output))
        
        # 2. Cross-attention to encoder output
        cross_attn_output, _ = self.cross_attn(
            query=x,
            key=encoder_output,
            value=encoder_output,
            mask=src_mask
        )
        x = self.norm2(x + self.dropout(cross_attn_output))
        
        # 3. Feed-forward network
        ffn_output = self.ffn(x)
        x = self.norm3(x + self.dropout(ffn_output))
        
        return x

# Test decoder block
d_model = 512
num_heads = 8
d_ff = 2048
src_seq_len = 12  # Encoder output length
tgt_seq_len = 8   # Decoder input length
batch_size = 2

decoder_block = DecoderBlock(d_model, num_heads, d_ff)
decoder_input = torch.randn(batch_size, tgt_seq_len, d_model)
encoder_output = torch.randn(batch_size, src_seq_len, d_model)

output = decoder_block(decoder_input, encoder_output)

print("✅ Decoder Block Test")
print(f"Decoder input shape: {decoder_input.shape}")
print(f"Encoder output shape: {encoder_output.shape}")
print(f"Decoder output shape: {output.shape}")
print(f"\n🎯 Complete decoder block working!")
print(f"   - Masked self-attention ✅")
print(f"   - Cross-attention to encoder ✅")
print(f"   - Feed-forward network ✅")

## 🚀 Complete Transformer Architecture

**Now let's put it all together!**

### 📐 Full Transformer:

```
SOURCE SEQUENCE          TARGET SEQUENCE
     ↓                         ↓
 Embeddings              Embeddings
     ↓                         ↓
 Pos Encoding            Pos Encoding
     ↓                         ↓
┌─────────┐             ┌─────────┐
│ Encoder │             │ Decoder │
│  Block  │             │  Block  │ ← Cross-attn to encoder
└─────────┘             └─────────┘
     ↓                         ↓
     ⋮                         ⋮
(Repeat N times)        (Repeat N times)
     ↓                         ↓
Encoder Output          Linear + Softmax
                              ↓
                        Output Probabilities
```

### 🎯 Typical Hyperparameters:

**Original Transformer (2017):**
- `d_model = 512`
- `num_heads = 8`
- `d_ff = 2048`
- `num_layers = 6`
- `vocab_size = 37,000`

**GPT-3 (2020):**
- `d_model = 12,288`
- `num_heads = 96`
- `d_ff = 49,152`
- `num_layers = 96`
- `vocab_size = 50,257`
- **Total: 175 billion parameters!**

In [ ]:
# Complete Transformer Model

class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        d_model=512,
        num_heads=8,
        num_encoder_layers=6,
        num_decoder_layers=6,
        d_ff=2048,
        max_seq_len=5000,
        dropout=0.1
    ):
        """
        Complete Transformer Model
        
        Args:
            src_vocab_size: Source vocabulary size
            tgt_vocab_size: Target vocabulary size
            d_model: Model dimension
            num_heads: Number of attention heads
            num_encoder_layers: Number of encoder blocks
            num_decoder_layers: Number of decoder blocks
            d_ff: Feed-forward dimension
            max_seq_len: Maximum sequence length
            dropout: Dropout probability
        """
        super(Transformer, self).__init__()
        
        # Embeddings
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)
        
        # Positional encoding
        self.pos_encoding = PositionalEncoding(d_model, max_seq_len, dropout)
        
        # Encoder stack
        self.encoder_layers = nn.ModuleList([
            EncoderBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_encoder_layers)
        ])
        
        # Decoder stack
        self.decoder_layers = nn.ModuleList([
            DecoderBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_decoder_layers)
        ])
        
        # Output layer
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model
        
    def encode(self, src, src_mask=None):
        """
        Encode source sequence
        
        Args:
            src: Source token IDs (batch_size, src_seq_len)
            src_mask: Source mask
        
        Returns:
            Encoder output (batch_size, src_seq_len, d_model)
        """
        # Embed and scale
        x = self.src_embedding(src) * math.sqrt(self.d_model)
        
        # Add positional encoding
        x = self.pos_encoding(x)
        
        # Pass through encoder layers
        for layer in self.encoder_layers:
            x = layer(x, src_mask)
        
        return x
    
    def decode(self, tgt, encoder_output, src_mask=None, tgt_mask=None):
        """
        Decode target sequence
        
        Args:
            tgt: Target token IDs (batch_size, tgt_seq_len)
            encoder_output: Encoder output (batch_size, src_seq_len, d_model)
            src_mask: Source mask
            tgt_mask: Target mask (for masking future positions)
        
        Returns:
            Decoder output (batch_size, tgt_seq_len, d_model)
        """
        # Embed and scale
        x = self.tgt_embedding(tgt) * math.sqrt(self.d_model)
        
        # Add positional encoding
        x = self.pos_encoding(x)
        
        # Pass through decoder layers
        for layer in self.decoder_layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        
        return x
    
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        """
        Forward pass
        
        Args:
            src: Source tokens (batch_size, src_seq_len)
            tgt: Target tokens (batch_size, tgt_seq_len)
            src_mask: Source mask
            tgt_mask: Target mask
        
        Returns:
            Output logits (batch_size, tgt_seq_len, tgt_vocab_size)
        """
        # Encode source
        encoder_output = self.encode(src, src_mask)
        
        # Decode target
        decoder_output = self.decode(tgt, encoder_output, src_mask, tgt_mask)
        
        # Project to vocabulary
        output = self.fc_out(decoder_output)
        
        return output

# Test complete transformer
src_vocab_size = 10000
tgt_vocab_size = 10000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048

transformer = Transformer(
    src_vocab_size=src_vocab_size,
    tgt_vocab_size=tgt_vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_encoder_layers=num_layers,
    num_decoder_layers=num_layers,
    d_ff=d_ff
)

# Count parameters
num_params = sum(p.numel() for p in transformer.parameters())

# Test with random data
batch_size = 2
src_seq_len = 10
tgt_seq_len = 8

src = torch.randint(0, src_vocab_size, (batch_size, src_seq_len))
tgt = torch.randint(0, tgt_vocab_size, (batch_size, tgt_seq_len))

output = transformer(src, tgt)

print("🎉 COMPLETE TRANSFORMER BUILT!\n")
print("="*70)
print(f"\n📊 Model Configuration:")
print(f"   - Model dimension (d_model): {d_model}")
print(f"   - Number of attention heads: {num_heads}")
print(f"   - Number of encoder layers: {num_layers}")
print(f"   - Number of decoder layers: {num_layers}")
print(f"   - Feed-forward dimension: {d_ff}")
print(f"   - Source vocabulary: {src_vocab_size:,}")
print(f"   - Target vocabulary: {tgt_vocab_size:,}")
print(f"\n💪 Total Parameters: {num_params:,}")
print(f"\n✅ Test Forward Pass:")
print(f"   - Source shape: {src.shape}")
print(f"   - Target shape: {tgt.shape}")
print(f"   - Output shape: {output.shape}")
print(f"\n🚀 This is the SAME architecture as the original 'Attention Is All You Need' paper!")
print(f"   You just built the foundation of modern AI! 🎉")

## 🎭 GPT vs BERT vs T5: How They Differ

**All use transformers, but with different architectures!**

### 🤖 GPT (Generative Pre-trained Transformer)

**Architecture:** Decoder-Only

```
Input: "The cat sat on"
   ↓
Embeddings + Positional Encoding
   ↓
┌──────────────────────────┐
│  Masked Self-Attention   │ ← Can only see previous words
│  Feed-Forward Network    │
│  (Repeat 96 layers)      │
└──────────────────────────┘
   ↓
Output: "the" (next word)
```

**Key Features:**
- ❌ **No encoder** (decoder-only)
- ✅ **Masked attention** (autoregressive)
- ✅ **Left-to-right** generation
- 🎯 **Objective:** Predict next token

**Models:**
- GPT-2 (1.5B parameters)
- GPT-3 (175B parameters)
- ChatGPT (GPT-3.5/4 with RLHF)
- GPT-4 (1.7T+ parameters, multimodal)

**Best For:**
- Text generation
- Completion tasks
- Conversational AI
- Creative writing

---

### 🔍 BERT (Bidirectional Encoder Representations from Transformers)

**Architecture:** Encoder-Only

```
Input: "The [MASK] sat on the mat"
   ↓
Embeddings + Positional Encoding
   ↓
┌──────────────────────────┐
│  Self-Attention          │ ← Sees ALL words (bidirectional)
│  Feed-Forward Network    │
│  (Repeat 12-24 layers)   │
└──────────────────────────┘
   ↓
Output: "cat" (fill in the mask)
```

**Key Features:**
- ✅ **Encoder-only** architecture
- ✅ **Bidirectional** (sees past AND future)
- ❌ **No masked attention** (not autoregressive)
- 🎯 **Objective:** Masked Language Modeling (MLM)

**Models:**
- BERT-Base (110M parameters)
- BERT-Large (340M parameters)
- RoBERTa (355M, improved BERT)
- DistilBERT (66M, faster)

**Best For:**
- Text classification
- Named entity recognition
- Question answering
- Understanding tasks

---

### 🔄 T5 (Text-to-Text Transfer Transformer)

**Architecture:** Encoder-Decoder (Full Transformer)

```
Input: "translate English to French: The cat"
   ↓
ENCODER (bidirectional)
   ↓
DECODER (autoregressive)
   ↓
Output: "Le chat"
```

**Key Features:**
- ✅ **Both encoder AND decoder**
- ✅ **Everything is text-to-text**
- ✅ **Unified framework** for all tasks
- 🎯 **Objective:** Span corruption + supervised tasks

**Models:**
- T5-Small (60M)
- T5-Base (220M)
- T5-Large (770M)
- T5-11B (11 billion)

**Best For:**
- Translation
- Summarization
- Question answering
- Any text-to-text task

---

### 📊 Quick Comparison:

| Feature | GPT | BERT | T5 |
|---------|-----|------|----|
| **Architecture** | Decoder-only | Encoder-only | Encoder-Decoder |
| **Attention** | Masked (left-to-right) | Bidirectional | Both |
| **Objective** | Next token prediction | Masked LM | Text-to-text |
| **Best For** | Generation | Understanding | Both |
| **Example** | ChatGPT | Search ranking | Translation |
| **Can generate?** | ✅ Yes | ❌ No | ✅ Yes |
| **Can understand?** | ✅ Yes | ✅ Yes (better) | ✅ Yes |

### 🌟 Modern Trends (2024-2025):

**Decoder-only models (GPT-style) are WINNING:**
- GPT-4, Claude, Gemini all use decoder-only
- Llama 3, Mistral, Phi-3 all decoder-only
- **Why?** Scales better, simpler architecture, strong at both generation AND understanding

**But BERT-style still used for:**
- Embeddings (sentence-transformers)
- Search and ranking
- Classification tasks

**T5-style for specific tasks:**
- Translation
- Summarization
- Structured outputs

## 🎯 Real AI Example: Building a Mini-Transformer

**Task:** Train a small transformer for sequence copying

**Goal:** Given `[1, 2, 3, 4]`, output `[1, 2, 3, 4]`

**Why This Matters:**
- Validates our transformer implementation
- Shows how transformers learn patterns
- Simpler than full translation, but same principles
- This is how ChatGPT training starts (next token prediction)

Let's build and train it!

In [ ]:
# Simplified Transformer for sequence copying

def generate_square_subsequent_mask(sz):
    """Generate mask for decoder (prevents looking at future tokens)"""
    mask = torch.triu(torch.ones(sz, sz), diagonal=1)
    mask = mask.masked_fill(mask == 1, float('-inf'))
    return mask

# Create a small transformer
vocab_size = 20  # Small vocabulary for demo
d_model = 128
num_heads = 4
num_layers = 2
d_ff = 512

model = Transformer(
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_encoder_layers=num_layers,
    num_decoder_layers=num_layers,
    d_ff=d_ff,
    dropout=0.1
)

# Generate training data (sequence copying)
def generate_data(batch_size, seq_len, vocab_size):
    """Generate random sequences for copying task"""
    # Source and target are the same (copying task)
    src = torch.randint(1, vocab_size, (batch_size, seq_len))
    tgt = src.clone()
    return src, tgt

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Training loop (just a few iterations for demo)
num_epochs = 50
batch_size = 32
seq_len = 8

print("🎓 Training Mini-Transformer on Sequence Copying Task...\n")
print("="*70)

model.train()
losses = []

for epoch in range(num_epochs):
    # Generate batch
    src, tgt = generate_data(batch_size, seq_len, vocab_size)
    
    # Create target input (shift right by 1)
    tgt_input = tgt[:, :-1]  # All but last token
    tgt_output = tgt[:, 1:]  # All but first token
    
    # Create target mask (prevent looking at future)
    tgt_mask = generate_square_subsequent_mask(tgt_input.size(1))
    
    # Forward pass
    output = model(src, tgt_input, tgt_mask=tgt_mask)
    
    # Calculate loss
    output = output.reshape(-1, vocab_size)
    tgt_output = tgt_output.reshape(-1)
    loss = criterion(output, tgt_output)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:3d}/{num_epochs} | Loss: {loss.item():.4f}")

print("\n✅ Training complete!")

# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(losses, linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Transformer Training Loss (Sequence Copying)', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Loss decreasing = Transformer is learning!")

In [ ]:
# Test the trained transformer

def test_transformer(model, test_seq):
    """
    Test transformer on a sequence
    """
    model.eval()
    
    with torch.no_grad():
        # Prepare input
        src = test_seq.unsqueeze(0)  # Add batch dimension
        
        # Encode
        encoder_output = model.encode(src)
        
        # Start with start token (0)
        tgt_tokens = [0]
        
        # Generate sequence autoregressively
        for _ in range(test_seq.size(0)):
            tgt = torch.LongTensor([tgt_tokens]).to(src.device)
            tgt_mask = generate_square_subsequent_mask(tgt.size(1))
            
            # Decode
            output = model.decode(tgt, encoder_output, tgt_mask=tgt_mask)
            output = model.fc_out(output)
            
            # Get next token (greedy)
            next_token = output[0, -1, :].argmax().item()
            tgt_tokens.append(next_token)
        
        return tgt_tokens[1:]  # Remove start token

# Test with several sequences
print("🧪 Testing Trained Transformer:\n")
print("="*70)

for i in range(5):
    # Generate random test sequence
    test_seq = torch.randint(1, vocab_size, (seq_len,))
    predicted = test_transformer(model, test_seq)
    
    print(f"\nTest {i+1}:")
    print(f"  Input:     {test_seq.tolist()}")
    print(f"  Predicted: {predicted}")
    
    # Check accuracy
    correct = sum([a == b for a, b in zip(test_seq.tolist(), predicted)])
    accuracy = correct / len(test_seq) * 100
    print(f"  Accuracy:  {accuracy:.1f}% ({correct}/{len(test_seq)} tokens)")

print("\n" + "="*70)
print("\n💡 The transformer learned to copy sequences!")
print("   This demonstrates the same principle as ChatGPT:")
print("   - Encoder understands the input")
print("   - Decoder generates output token by token")
print("   - Attention connects relevant parts")
print("\n🚀 You just built and trained a working Transformer!")

## 🎯 Interactive Exercises

Test your understanding of the Transformer architecture!

### Exercise 1: Architecture Design

**Question:** You want to build a model for text classification (sentiment analysis). Which architecture should you use?

A) Full Transformer (Encoder + Decoder)  
B) Encoder-only (BERT-style)  
C) Decoder-only (GPT-style)  

**Explain your reasoning.**

<details>
<summary>📖 Click for solution</summary>

**Answer: B) Encoder-only (BERT-style)**

**Reasoning:**

1. **Text Classification Needs:**
   - Understand the ENTIRE input
   - No need to generate output sequences
   - Single prediction (positive/negative)

2. **Why Encoder-only?**
   - ✅ Bidirectional attention sees full context
   - ✅ Better understanding of meaning
   - ✅ No decoder overhead (faster)
   - ✅ BERT, RoBERTa designed for this!

3. **Why NOT the others?**
   - **Full Transformer**: Overkill, decoder not needed
   - **Decoder-only**: Can work but less efficient for classification

**In Practice:**
- Sentiment analysis: BERT ✅
- Text generation: GPT ✅
- Translation: T5 (full transformer) ✅
</details>

### Exercise 2: Masking Understanding

**Question:** Why does the decoder use masked self-attention but the encoder doesn't?

Draw or explain the attention patterns for:
1. Encoder self-attention
2. Decoder masked self-attention

<details>
<summary>📖 Click for solution</summary>

**Answer:**

**Encoder (No Masking):**
```
Input: "The cat sat"

Attention Matrix (all positions can see all positions):
       The  cat  sat
The  [ ✅   ✅   ✅ ]  ← Can see all words
cat  [ ✅   ✅   ✅ ]  ← Can see all words
sat  [ ✅   ✅   ✅ ]  ← Can see all words

Why? Encoder just understands input, not generating.
```

**Decoder (With Masking):**
```
Output: "Le chat dort"

Attention Matrix (can only see previous positions):
       Le  chat dort
Le   [ ✅   ❌   ❌ ]  ← Only sees itself
chat [ ✅   ✅   ❌ ]  ← Sees Le, chat
dort [ ✅   ✅   ✅ ]  ← Sees all previous

Why? During training, we know full output.
Must prevent cheating by seeing future!
```

**Key Insight:**
- **Encoder**: Understands complete input → no masking needed
- **Decoder**: Generates word-by-word → must mask future to prevent cheating

**During Inference:**
- Masking naturally enforced (we don't have future tokens yet!)
- Generate one token at a time
- This is how ChatGPT generates responses!
</details>

### Exercise 3: Scaling Analysis

**Challenge:** Calculate the number of parameters in a transformer

**Given:**
- `d_model = 512`
- `num_heads = 8`
- `d_ff = 2048`
- `vocab_size = 50,000`
- `num_encoder_layers = 6`
- `num_decoder_layers = 6`

**Calculate (approximately):**
1. Parameters in one multi-head attention layer
2. Parameters in one feed-forward network
3. Total model parameters

**Hint:** 
- Multi-head attention: 4 linear layers (Q, K, V, O) of size `d_model × d_model`
- FFN: 2 linear layers (`d_model × d_ff` and `d_ff × d_model`)
- Don't forget embeddings!

In [ ]:
# Solution: Parameter Calculation

d_model = 512
num_heads = 8
d_ff = 2048
vocab_size = 50000
num_encoder_layers = 6
num_decoder_layers = 6

print("📊 Transformer Parameter Calculation\n")
print("="*70)

# 1. Multi-Head Attention Parameters
# Q, K, V, O projections: 4 × (d_model × d_model)
mha_params = 4 * (d_model * d_model)
print(f"\n1️⃣ Multi-Head Attention (one layer):")
print(f"   4 linear layers × ({d_model} × {d_model}) = {mha_params:,} params")

# 2. Feed-Forward Network Parameters
# Two linear layers: (d_model × d_ff) + (d_ff × d_model)
ffn_params = (d_model * d_ff) + (d_ff * d_model)
print(f"\n2️⃣ Feed-Forward Network (one layer):")
print(f"   ({d_model} × {d_ff}) + ({d_ff} × {d_model}) = {ffn_params:,} params")

# 3. One Encoder Block
encoder_block_params = mha_params + ffn_params
print(f"\n3️⃣ One Encoder Block:")
print(f"   MHA + FFN = {encoder_block_params:,} params")

# 4. One Decoder Block (has 2 attention layers!)
decoder_block_params = 2 * mha_params + ffn_params  # Masked + cross attention
print(f"\n4️⃣ One Decoder Block:")
print(f"   2×MHA + FFN = {decoder_block_params:,} params")

# 5. All Encoder Layers
total_encoder_params = num_encoder_layers * encoder_block_params
print(f"\n5️⃣ All Encoder Layers ({num_encoder_layers} layers):")
print(f"   {num_encoder_layers} × {encoder_block_params:,} = {total_encoder_params:,} params")

# 6. All Decoder Layers
total_decoder_params = num_decoder_layers * decoder_block_params
print(f"\n6️⃣ All Decoder Layers ({num_decoder_layers} layers):")
print(f"   {num_decoder_layers} × {decoder_block_params:,} = {total_decoder_params:,} params")

# 7. Embeddings (source + target)
embedding_params = 2 * (vocab_size * d_model)
print(f"\n7️⃣ Embeddings (source + target):")
print(f"   2 × ({vocab_size:,} × {d_model}) = {embedding_params:,} params")

# 8. Output Layer
output_params = d_model * vocab_size
print(f"\n8️⃣ Output Layer:")
print(f"   {d_model} × {vocab_size:,} = {output_params:,} params")

# Total (approximate, ignoring layer norm and biases)
total_params = (
    total_encoder_params + 
    total_decoder_params + 
    embedding_params + 
    output_params
)

print("\n" + "="*70)
print(f"\n🎯 TOTAL PARAMETERS (approximate): {total_params:,}")
print(f"   That's about {total_params / 1e6:.1f} Million parameters!")

print("\n📊 Parameter Breakdown:")
print(f"   Embeddings:    {embedding_params:,} ({embedding_params/total_params*100:.1f}%)")
print(f"   Encoder:       {total_encoder_params:,} ({total_encoder_params/total_params*100:.1f}%)")
print(f"   Decoder:       {total_decoder_params:,} ({total_decoder_params/total_params*100:.1f}%)")
print(f"   Output:        {output_params:,} ({output_params/total_params*100:.1f}%)")

print("\n💡 Insights:")
print("   - Most parameters in embeddings + output (vocab size!)")
print("   - GPT-3 uses similar architecture, but MUCH larger")
print(f"   - GPT-3: 175 BILLION params ({175000 / (total_params/1e6):.0f}x larger!)")
print("\n🚀 Larger models = More capacity = Better performance!")

## 🎉 Key Takeaways

**You just mastered the Transformer architecture!**

### 1️⃣ **Transformer Components**
   - ✅ Multi-Head Self-Attention (parallel attention patterns)
   - ✅ Positional Encoding (adds order information)
   - ✅ Feed-Forward Networks (processes each position)
   - ✅ Residual Connections + Layer Norm (stable training)

### 2️⃣ **Encoder-Decoder Architecture**
   - ✅ Encoder: Bidirectional understanding of input
   - ✅ Decoder: Autoregressive generation of output
   - ✅ Cross-Attention: Links source and target
   - ✅ Masking: Prevents cheating during training

### 3️⃣ **Modern Variants**
   - ✅ **GPT (Decoder-only)**: Text generation, ChatGPT
   - ✅ **BERT (Encoder-only)**: Text understanding, classification
   - ✅ **T5 (Full Transformer)**: Text-to-text, translation

### 4️⃣ **Why Transformers Won**
   - ✅ Parallelizable (faster training)
   - ✅ Long-range dependencies (no forgetting)
   - ✅ Scalable (works with billions of parameters)
   - ✅ Transfer learning (pre-train once, use everywhere)

---

## 🌟 Real-World Impact (2024-2025)

**Every AI you use is built on Transformers:**

### 🤖 **Language Models**
- **ChatGPT**: GPT architecture (decoder-only)
- **Claude**: Constitutional AI transformer
- **Gemini**: Google's multimodal transformer
- **GPT-4**: 1.7T+ parameters, decoder-only

### 🎨 **Vision Models**
- **ViT (Vision Transformer)**: Images as token sequences
- **DALL-E**: Text → Image using transformers
- **Stable Diffusion**: Transformer-based diffusion

### 💻 **Code Models**
- **GitHub Copilot**: Code generation transformer
- **CodeLlama**: Specialized for programming
- **AlphaCode**: Competitive programming AI

### 🔬 **Science**
- **AlphaFold**: Protein structure prediction
- **ESM**: Protein language model
- **MolFormer**: Molecular property prediction

### 🎵 **Audio**
- **Whisper**: Speech recognition transformer
- **MusicGen**: Music generation
- **AudioLM**: Audio synthesis

---

## 📊 Scaling Laws

**Bigger Transformers = Better Performance:**

| Model | Year | Params | Architecture | What Changed |
|-------|------|--------|--------------|-------------|
| Transformer | 2017 | 65M | Enc-Dec | Invented attention |
| BERT | 2018 | 340M | Enc-only | Bidirectional understanding |
| GPT-2 | 2019 | 1.5B | Dec-only | Impressive generation |
| GPT-3 | 2020 | 175B | Dec-only | Few-shot learning |
| ChatGPT | 2022 | 175B+ | Dec-only | + RLHF |
| GPT-4 | 2023 | 1.7T+ | Dec-only | Multimodal |

**Key Insight:** Same architecture, just BIGGER!

---

## 🎯 What You Can Build Now

**With this knowledge:**
1. ✅ Understand how ChatGPT works internally
2. ✅ Fine-tune BERT for classification tasks
3. ✅ Use GPT-2/3 for text generation
4. ✅ Build custom transformers for specific tasks
5. ✅ Read and understand transformer research papers

**Next:** Day 3 - Working with LLMs (fine-tuning, APIs, RAG)!

---

**💬 Remember:**

*"Attention Is All You Need" wasn't just a paper title - it was a prediction. In 2017, nobody expected transformers to completely replace RNNs, CNNs, and every other architecture. But here we are in 2025, and transformers power EVERYTHING. You now understand the architecture behind this revolution!"* 🚀

## 🚀 Next Steps

**Before Day 3, explore:**
1. Modify the transformer architecture (change layers, heads)
2. Visualize attention weights from the trained model
3. Try different sequence tasks (reversal, sorting)
4. Read the original "Attention Is All You Need" paper

**Coming Next:**
- **Day 3:** Working with Modern LLMs
  - Using OpenAI API and HuggingFace
  - Fine-tuning GPT-2 for custom text generation
  - Building a RAG system with embeddings
  - Prompt engineering for better results
  - Deploying LLM applications

---

**📚 Deep Dive Resources:**
- "Attention Is All You Need" (Vaswani et al., 2017)
- The Illustrated Transformer (Jay Alammar)
- The Annotated Transformer (Harvard NLP)
- HuggingFace Transformers Documentation

---

*You just built the architecture that powers ChatGPT, Claude, and every modern AI! 🎉*

**See you on Day 3 for hands-on LLM applications!** 🚀